In [1]:
import numpy as np
from keras.utils import np_utils
from keras.layers import Activation, Dense, Dropout, Embedding, LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import os


C:\Users\apple\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# data
# politics : 0
# science : 1
# sports : 2
DATA = [ #sorry, I can't show you the data for copyright or something...
 
    
]

In [3]:
# split (labels and news)
data_X = []
data_y = []
for data in DATA:
    data_X.append(data[0]) 
    data_y.append(data[1])

In [4]:
c_0 = 0
c_1 = 0
c_2 = 0
for d in data_y:
    if d == 0:
        c_0 += 1
    elif d == 1:
        c_1 += 1
    else:
        c_2 += 1
        
print('politics : ' ,c_0)
print('science : ' ,c_1)
print('sports : ' ,c_2)
print('total : ' ,c_0 + c_1 + c_2)

politics :  1000
science :  1000
sports :  1000
total :  3000


In [5]:
def flatten(data):
    for item in data:
        if hasattr(item, '__iter__'):
            for element in flatten(item):
                yield element
        else:
            yield item

flatten(data_y)
data_y = np.array(data_y)
data_y = np_utils.to_categorical(data_y)
print('data_y_shape : ', data_y.shape)


data_y_shape :  (3000, 3)


In [6]:
#sentence to words
import re
import collections


def sentence2words(sentence):
    stopwords =["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself",
                "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
                "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these",
                "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", 
                "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", 
                "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", 
                "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again",
                "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each",
                "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", 
                "too", "very",  "can", "will", "just",  "should", "now", "s", "t", "don", "didn", "aren", "isn", "can", "re", "ll", "ve"]
    sentence = sentence.lower() # 小文字化
    sentence = sentence.replace("\n", "") # 改行削除
    sentence = re.sub(re.compile(r"[!-\?()' ‘’.,;/:-@[-`{-~]"), " ", sentence) # 記号をスペースに置き換え
    sentence = sentence.split(" ") # スペースで区切る
    sentence_words = []
    for word in sentence:
        if (re.compile(r"^.*[0-9]+.*$").fullmatch(word) is not None): # 数字が含まれるものは除外
            continue
        if word in stopwords: # ストップワードに含まれるものは除外
            continue
        sentence_words.append(word)        
    return sentence_words
 
# word to index
word_to_index = {}
index_to_word = {}
num_recs = 0
maxlen = 0
word_freqs = collections.Counter()
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    maxlen =max(maxlen, len(sentence_words))
    for word in sentence_words:
        word_freqs[word] += 1
    num_recs += 1
    
max_features = 4000
vocab_size = min(max_features, len(word_freqs)) + 2
word_to_index = {x[0]: i+2 for i, x in
              enumerate(word_freqs.most_common(max_features))}
word_to_index["PAD"] = 0
word_to_index["UNK"] = 1
index_to_word = {v: k for k, v in word_to_index.items()}

    

In [7]:
# number of headlines
print('num_recs : ', num_recs)

num_recs :  3000


In [8]:
# number of words in one headline
print('maxlen : ', maxlen)

maxlen :  25


In [9]:
print('word_freqs : ' ,len(word_freqs))

word_freqs :  6832


In [10]:
# 0 = padding
# 1 = unknown
# 2 = stop words
print('word_to_index : ', len(word_to_index))

word_to_index :  4002


In [11]:
# headline to index
data_X_vec = []
for sentence in data_X:
    sentence_words = sentence2words(sentence)
    sentence_ids = []
    for word in sentence_words:
        if word not in word_to_index:
            sentence_ids.append(1)
            continue
        sentence_ids.append(word_to_index[word])
    data_X_vec.append(sentence_ids)
   
 

In [12]:
MAXLEN = 20
data_X = sequence.pad_sequences(data_X_vec, maxlen=MAXLEN)

In [48]:
import collections
embeddings_index = collections.defaultdict(int)
GLOVE_DIR = os.path.join('Downloads', 'glove.6B')
f = open(os.path.join(GLOVE_DIR, 'glove.6B.300d.txt'), "r", encoding="utf-8" )
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [49]:
EMBEDDING_DIM = 300
X = np.zeros((data_X.shape[0], EMBEDDING_DIM))
for i in range(data_X.shape[0]):
    E = np.zeros((EMBEDDING_DIM, MAXLEN))
    words = [index_to_word[wid] for wid in data_X[i].tolist()]
    for j in range(MAXLEN):
        #if j in embeddings_index:
        E[:, j] = embeddings_index[words[j]]
    X[i, :] = np.sum(E, axis=1)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, data_y, test_size=0.2, random_state=0)

In [51]:
print('X_train : ', X_train.shape)
print('X_test : ', X_test.shape)
print('y_train : ', y_train.shape)
print('y_test : ', y_test.shape)

X_train :  (2400, 300)
X_test :  (600, 300)
y_train :  (2400, 3)
y_test :  (600, 3)


In [54]:
model = Sequential()
model.add(Dense(32, input_dim=EMBEDDING_DIM, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(3, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 32)                9632      
_________________________________________________________________
dropout_10 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_20 (Dense)             (None, 3)                 99        
Total params: 9,731
Trainable params: 9,731
Non-trainable params: 0
_________________________________________________________________


In [55]:
history = model.fit(X_train, y_train, batch_size=20,
                    epochs=5,
                    validation_data=(X_test, y_test))

Train on 2400 samples, validate on 600 samples
Epoch 1/5
2400/2400 [==============================] - 1s 298us/step - loss: 0.7688 - acc: 0.6912 - val_loss: 0.4164 - val_acc: 0.8550
Epoch 2/5
2400/2400 [==============================] - 0s 112us/step - loss: 0.4470 - acc: 0.8371 - val_loss: 0.3727 - val_acc: 0.8750
Epoch 3/5
2400/2400 [==============================] - 0s 111us/step - loss: 0.3728 - acc: 0.8608 - val_loss: 0.3607 - val_acc: 0.8800
Epoch 4/5
2400/2400 [==============================] - 0s 122us/step - loss: 0.3187 - acc: 0.8812 - val_loss: 0.3511 - val_acc: 0.8900
Epoch 5/5
2400/2400 [==============================] - 0s 109us/step - loss: 0.2900 - acc: 0.8925 - val_loss: 0.3435 - val_acc: 0.8850


In [56]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test score: {:.3f}, accuracy: {:.3f}".format(score[0], score[1]))

600/600 [==============================] - 0s 59us/step
Test score: 0.344, accuracy: 0.885
